# Contents:

**Part 0 : Environnement Setup**
 - <a href="#C1">Importation of libraries</a>
 - <a href="#C2">Dataset Import</a>
 
**Part 1 : Data Cleaning**
 - <a href="#C3">Dataset Overview</a>
 - <a href="#C4">Duplicates</a>
 - <a href="#C5">Outliers</a>
 - <a href="#C6">Generation of new files</a>

# Part 0 : Environnement Setup

# <a name="C1">Importation of libraries</a>

In [1]:
# Importing libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import missingno as msno
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
import statsmodels.api as sm
import datetime as dt

import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import seaborn as sns
sns.set()

C:\Users\najwa\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\najwa\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
# Dislaying versions of libraries
print("pandas version : " + pd.__version__)
print("numpy version : " + np.__version__)
print("matplotlib version : " + mp.__version__)
print("seaborn version : " + sns.__version__)

pandas version : 1.4.2
numpy version : 1.21.5
matplotlib version : 3.5.1
seaborn version : 0.11.2


# <a name="C2">Dataset Import</a>

In [3]:
# Importing the file
produits = pd.read_csv('produits.csv')
transactions = pd.read_csv('transactions.csv')
magasins = pd.read_csv('magasins.csv')

In [4]:
#Création de listes
df_liste = [produits, transactions, magasins]
df_noms= ['produits', 'transactions', 'magasins']

# Part 1 : Data Cleaning

# <a name="C3">Dataset Overview</a>

In [5]:
# Displaying first rows of each file
for df, nom in zip(df_liste, df_noms):
    print(nom, ':')
    display(df.head())
    print('\n')

produits :


,product_id,category,is_marketplace_product,price
0,1,Chaise,False,70
1,2,Chaise,False,351
2,4,Chaise,False,188
3,5,Chaise,False,323
4,6,Chaise,False,236




transactions :


,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity
0,2021-03-01,1,1,67,4359,1
1,2021-03-01,1,1,673,840,2
2,2021-03-01,1,1,529,186,1
3,2021-03-01,1,1,929,221,1
4,2021-03-01,1,1,900,654,2




magasins :


,store_id,channel,country_iso,is_marketplace
0,1,Web,FR,Non
1,9,Web,FR,Oui
2,2,Store,FR,Non
3,3,Store,FR,Non
4,4,Store,FR,Non


In [6]:
# Displaying shape of each file
for df, nom in zip(df_liste, df_noms):
    print(nom, ':', df.shape)

produits : (1200, 4)
transactions : (28787, 6)
magasins : (9, 4)


In [7]:
# Displaying NaNs and types for each column in each file
for df, nom in zip(df_liste, df_noms):
    print(nom, ':')
    display(df.info())

produits :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   product_id              1200 non-null   int64 
 1   category                1200 non-null   object
 2   is_marketplace_product  1200 non-null   bool  
 3   price                   1200 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 29.4+ KB


None

transactions :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28787 entries, 0 to 28786
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_date  28787 non-null  object
 1   transaction_id    28787 non-null  int64 
 2   store_id          28787 non-null  int64 
 3   product_id        28787 non-null  int64 
 4   eur_amount        28787 non-null  int64 
 5   quantity          28787 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.3+ MB


None

magasins :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   store_id        9 non-null      int64 
 1   channel         9 non-null      object
 2   country_iso     9 non-null      object
 3   is_marketplace  9 non-null      object
dtypes: int64(1), object(3)
memory usage: 416.0+ bytes


None

In [8]:
# Displaying descriptive info of the file
for df, nom in zip(df_liste, df_noms):
    print(nom, ':')
    display(df.describe(include='all'))

produits :


,product_id,category,is_marketplace_product,price
count,1200.000000,1200,1200,1200.000000
unique,NaN,3,2,NaN
top,NaN,Canapé,False,NaN
freq,NaN,415,1000,NaN
mean,433.833333,NaN,NaN,1600.341667
std,303.807595,NaN,NaN,2239.511508
min,1.000000,NaN,NaN,31.000000
25%,150.750000,NaN,NaN,200.000000
50%,400.500000,NaN,NaN,371.000000
75%,700.250000,NaN,NaN,2477.500000


transactions :


,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity
count,28787,28787.00000,28787.000000,28787.000000,28787.000000,28787.000000
unique,28,NaN,NaN,NaN,NaN,NaN
top,2021-03-06,NaN,NaN,NaN,NaN,NaN
freq,1152,NaN,NaN,NaN,NaN,NaN
mean,NaN,4995.23000,3.725814,471.757495,2428.160559,2.376698
std,NaN,2884.82651,2.832335,297.764812,3715.727897,1.989910
min,NaN,1.00000,1.000000,1.000000,31.000000,1.000000
25%,NaN,2497.00000,1.000000,193.000000,385.000000,1.000000
50%,NaN,4998.00000,3.000000,463.000000,920.000000,1.000000
75%,NaN,7481.00000,6.000000,732.000000,3081.000000,3.000000


magasins :


,store_id,channel,country_iso,is_marketplace
count,9.000000,9,9,9
unique,NaN,2,3,2
top,NaN,Store,FR,Non
freq,NaN,7,5,8
mean,5.000000,NaN,NaN,NaN
std,2.738613,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN
25%,3.000000,NaN,NaN,NaN
50%,5.000000,NaN,NaN,NaN
75%,7.000000,NaN,NaN,NaN


# <a name="C4">Duplicates</a>

In [9]:
# Searching duplicates in the dataset
for df, nom in zip(df_liste, df_noms):
    print(nom, df.duplicated().sum())

produits 0
transactions 24
magasins 0


In [10]:
# Removing duplicates in the dataset transactions
transactions.drop_duplicates(inplace=True)
transactions.duplicated().sum()

0

In [11]:
# Checking unicity of identifiers for dataset produits
produits['product_id'].duplicated().sum()

200

In [12]:
# Checking unicity of identifiers for dataset produits
magasins['store_id'].duplicated().sum()

0

In [13]:
# Searching duplicates in the dataset produits
produits_temp = produits[['product_id', 'category']]
produits_temp.duplicated().sum()

63

In [14]:
# Searching duplicates in the dataset produits
produits_temp = produits[['product_id', 'is_marketplace_product']]
produits_temp.duplicated().sum()

0

In [15]:
# Assigning unique product id - if marketplace, starting with 2, if not, starting with 1
produits['product_id'] = np.where(produits.is_marketplace_product, produits['product_id'] + 2000000, produits['product_id'] + 1000000)

In [16]:
# Checking unicity of identifiers for dataset produits
produits['product_id'].duplicated().sum()

0

In [17]:
# Checking unicity of identifiers for dataset produits
produits.describe()

,product_id,price
count,1.200000e+03,1200.000000
mean,1.167100e+06,1600.341667
std,3.726843e+05,2239.511508
min,1.000001e+06,31.000000
25%,1.000301e+06,200.000000
50%,1.000600e+06,371.000000
75%,1.000900e+06,2477.500000
max,2.000200e+06,7986.000000


In [18]:
# Analysing further dataset Magasins
magasins_heat = pd.pivot_table(magasins, values = ['is_marketplace'], index =['store_id','country_iso'], columns=['channel'], aggfunc=np.sum)
magasins_heat

is_marketplace     
channel                       Store  Web
store_id country_iso                    
1        FR                     NaN  Non
2        FR                     Non  NaN
3        FR                     Non  NaN
4        FR                     Non  NaN
5        ES                     Non  NaN
6        ES                     Non  NaN
7        IT                     Non  NaN
8        IT                     Non  NaN
9        FR                     NaN  Oui

In [19]:
# Assigning unique product id - if marketplace, starting with 2, if not, starting with 1
transactions['product_id'] = np.where(transactions.store_id == 9, transactions['product_id'] + 2000000, transactions['product_id'] + 1000000)

In [20]:
# Checking unicity of identifiers for dataset produits
transactions.describe()

,transaction_id,store_id,product_id,eur_amount,quantity
count,28763.000000,28763.000000,2.876300e+04,28763.000000,28763.000000
mean,4994.997462,3.725272,1.074977e+06,2428.953864,2.377256
std,2884.761478,2.831837,2.624905e+05,3716.734278,1.990166
min,1.000000,1.000000,1.000001e+06,31.000000,1.000000
25%,2497.000000,1.000000,1.000272e+06,386.000000,1.000000
50%,4998.000000,3.000000,1.000544e+06,920.000000,1.000000
75%,7481.000000,6.000000,1.000810e+06,3081.000000,3.000000
max,10000.000000,9.000000,2.000199e+06,39840.000000,12.000000


In [21]:
# Displaying shape of each file
for df, nom in zip(df_liste, df_noms):
    print(nom, ':', df.shape)

produits : (1200, 4)
transactions : (28763, 6)
magasins : (9, 4)


# <a name="C5">Outliers</a>

In [22]:
# Displaying variables statistics for each quantitative columns
for df, nom in zip(df_liste, df_noms):
    print(nom, ':')
    display(df.describe())

produits :


,product_id,price
count,1.200000e+03,1200.000000
mean,1.167100e+06,1600.341667
std,3.726843e+05,2239.511508
min,1.000001e+06,31.000000
25%,1.000301e+06,200.000000
50%,1.000600e+06,371.000000
75%,1.000900e+06,2477.500000
max,2.000200e+06,7986.000000


transactions :


,transaction_id,store_id,product_id,eur_amount,quantity
count,28763.000000,28763.000000,2.876300e+04,28763.000000,28763.000000
mean,4994.997462,3.725272,1.074977e+06,2428.953864,2.377256
std,2884.761478,2.831837,2.624905e+05,3716.734278,1.990166
min,1.000000,1.000000,1.000001e+06,31.000000,1.000000
25%,2497.000000,1.000000,1.000272e+06,386.000000,1.000000
50%,4998.000000,3.000000,1.000544e+06,920.000000,1.000000
75%,7481.000000,6.000000,1.000810e+06,3081.000000,3.000000
max,10000.000000,9.000000,2.000199e+06,39840.000000,12.000000


magasins :


,store_id
count,9.000000
mean,5.000000
std,2.738613
min,1.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,9.000000


In [23]:
# At first sight, no "visible" outliers. The range for each variable is possible.
# Let's consider some new variables to identify possible outliers

In [24]:
# Merging files 
dataset = pd.merge(transactions, produits, on='product_id', how='left', indicator=True)

In [25]:
# Checking descrtiptive details of resulting file
dataset.describe(include='all')

,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity,category,is_marketplace_product,price,_merge
count,28763,28763.000000,28763.000000,2.876300e+04,28763.000000,28763.000000,28763,28763,28763.000000,28763
unique,28,NaN,NaN,NaN,NaN,NaN,3,2,NaN,1
top,2021-03-06,NaN,NaN,NaN,NaN,NaN,Canapé,False,NaN,both
freq,1151,NaN,NaN,NaN,NaN,NaN,9879,26620,NaN,28763
mean,NaN,4994.997462,3.725272,1.074977e+06,2428.953864,2.377256,NaN,NaN,1584.569551,NaN
std,NaN,2884.761478,2.831837,2.624905e+05,3716.734278,1.990166,NaN,NaN,2224.638164,NaN
min,NaN,1.000000,1.000000,1.000001e+06,31.000000,1.000000,NaN,NaN,31.000000,NaN
25%,NaN,2497.000000,1.000000,1.000272e+06,386.000000,1.000000,NaN,NaN,199.000000,NaN
50%,NaN,4998.000000,3.000000,1.000544e+06,920.000000,1.000000,NaN,NaN,371.000000,NaN
75%,NaN,7481.000000,6.000000,1.000810e+06,3081.000000,3.000000,NaN,NaN,2476.000000,NaN


In [26]:
# Computing a new variable
dataset['eur_amount_comp'] = dataset['quantity']*dataset['price']

In [27]:
# Checking descrtiptive details of resulting file
dataset.describe()

,transaction_id,store_id,product_id,eur_amount,quantity,price,eur_amount_comp
count,28763.000000,28763.000000,2.876300e+04,28763.000000,28763.000000,28763.000000,28763.000000
mean,4994.997462,3.725272,1.074977e+06,2428.953864,2.377256,1584.569551,2428.953864
std,2884.761478,2.831837,2.624905e+05,3716.734278,1.990166,2224.638164,3716.734278
min,1.000000,1.000000,1.000001e+06,31.000000,1.000000,31.000000,31.000000
25%,2497.000000,1.000000,1.000272e+06,386.000000,1.000000,199.000000,386.000000
50%,4998.000000,3.000000,1.000544e+06,920.000000,1.000000,371.000000,920.000000
75%,7481.000000,6.000000,1.000810e+06,3081.000000,3.000000,2476.000000,3081.000000
max,10000.000000,9.000000,2.000199e+06,39840.000000,12.000000,7986.000000,39840.000000


In [28]:
# Removing the column merge used to check correct merge done and new variables used for check
dataset = dataset.drop(['eur_amount_comp','_merge'],axis=1)
dataset.head()

,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity,category,is_marketplace_product,price
0,2021-03-01,1,1,1000067,4359,1,Canapé,False,4359
1,2021-03-01,1,1,1000673,840,2,Miroir,False,420
2,2021-03-01,1,1,1000529,186,1,Miroir,False,186
3,2021-03-01,1,1,1000929,221,1,Miroir,False,221
4,2021-03-01,1,1,1000900,654,2,Miroir,False,327


In [29]:
# We can see that eur_amount and eur_amount_comp has excatly same statistics. We can assume they are the same, which comfort the content for eur_amount.

# <a name="C6">Generation of new dataset</a>

In [30]:
# Merging files 
dataset = pd.merge(dataset, magasins, on='store_id', how='left', indicator=True)

In [31]:
# Checking descrtiptive details of resulting file
dataset.describe(include='all')

,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity,category,is_marketplace_product,price,channel,country_iso,is_marketplace,_merge
count,28763,28763.000000,28763.000000,2.876300e+04,28763.000000,28763.000000,28763,28763,28763.000000,28763,28763,28763,28763
unique,28,NaN,NaN,NaN,NaN,NaN,3,2,NaN,2,3,2,1
top,2021-03-06,NaN,NaN,NaN,NaN,NaN,Canapé,False,NaN,Store,FR,Non,both
freq,1151,NaN,NaN,NaN,NaN,NaN,9879,26620,NaN,15157,19902,26620,28763
mean,NaN,4994.997462,3.725272,1.074977e+06,2428.953864,2.377256,NaN,NaN,1584.569551,NaN,NaN,NaN,NaN
std,NaN,2884.761478,2.831837,2.624905e+05,3716.734278,1.990166,NaN,NaN,2224.638164,NaN,NaN,NaN,NaN
min,NaN,1.000000,1.000000,1.000001e+06,31.000000,1.000000,NaN,NaN,31.000000,NaN,NaN,NaN,NaN
25%,NaN,2497.000000,1.000000,1.000272e+06,386.000000,1.000000,NaN,NaN,199.000000,NaN,NaN,NaN,NaN
50%,NaN,4998.000000,3.000000,1.000544e+06,920.000000,1.000000,NaN,NaN,371.000000,NaN,NaN,NaN,NaN
75%,NaN,7481.000000,6.000000,1.000810e+06,3081.000000,3.000000,NaN,NaN,2476.000000,NaN,NaN,NaN,NaN


In [32]:
# Adding a column transaction_week with week number
dataset['transaction_date'] = pd.to_datetime(dataset['transaction_date'], errors ='coerce')
dataset['transaction_week'] = dataset['transaction_date'].dt.isocalendar().week

In [33]:
# Checking the result
dataset.tail()

,transaction_date,transaction_id,store_id,product_id,eur_amount,quantity,category,is_marketplace_product,price,channel,country_iso,is_marketplace,_merge,transaction_week
28758,2021-03-28,9998,4,1000268,8235,3,Canapé,False,2745,Store,FR,Non,both,12
28759,2021-03-28,9999,1,1000979,3983,1,Canapé,False,3983,Web,FR,Non,both,12
28760,2021-03-28,9999,1,1000818,748,4,Chaise,False,187,Web,FR,Non,both,12
28761,2021-03-28,10000,1,1000445,360,2,Chaise,False,180,Web,FR,Non,both,12
28762,2021-03-28,10000,1,1000762,10816,2,Canapé,False,5408,Web,FR,Non,both,12


In [34]:
# Storing the dataset locally
dataset.to_csv('dataset.csv', index=False)

The end of data cleaning